In [19]:
!pip install langchain-groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.5/103.5 kB 5.8 MB/s eta 0:00:00


In [1]:
from langchain_community.document_loaders import PyPDFLoader
import os

In [2]:
from typing import List, Optional
import json

from pydantic import BaseModel, Field
from groq import Groq

groq = Groq(api_key='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')

In [3]:
import os

def list_files_in_directory(directory_path):
    try:
        files_list = []
        
        # Walk through the directory and its subdirectories
        for root, _, files in os.walk(directory_path):
            if '.ipynb_checkpoints' in root:
                continue
            for file in files:
                full_path = os.path.join(root, file)
                files_list.append(full_path)
        
        return files_list
    except Exception as e:
        return str(e)

directory_path = './DORIS/'
files = list_files_in_directory(directory_path)


In [4]:
class Extraction(BaseModel):
    product_name: str = Field(description="name of the product")
    manufacturer: str = Field(description="name of the manufacturer")
    address: str = Field(description="address of the manufacturer")
    # use_of_the_chemical: str = Field(description="comma separated string values for recommended use of the chemical and restrictions on use")


In [33]:
import os

def is_pdf_and_starts_with_sds(file_path):
    if not file_path.lower().endswith('.pdf'):
        print(f'{file_path} : is not PDF')
    
    file_name = os.path.basename(file_path)
    if file_name.startswith('SDS'):
        print(f'{file_path} : is an sds document.')
        try:
            loader = PyPDFLoader(file_path)
            pages = loader.load_and_split()
            info_page = pages[0].page_content
            chat_completion = groq.chat.completions.create(
                messages=[
                    {
                        "role": "system",
                        "content": "You are an information extractor agent. Your task is to extract specific pieces of information from the given text and outputs in JSON.. If not explicitly provided do not guess. Return empty string for respective key if data not extracted.\n"
                        f" The JSON object must use the schema: {json.dumps(Extraction.model_json_schema(), indent=2)}",
                    },
                    {
                        "role": "user",
                        "content": f"Given text : \n {info_page}",
                    },
                ],
                model="llama3-8b-8192",
                temperature=0,
                stream=False,
                response_format={"type": "json_object"},
        )
            return Extraction.model_validate_json(chat_completion.choices[0].message.content)
        except:
            print(f'Some error in parsing pdf : {file_path}')
    else:
        print(f'{file_path} : File is not SDS file.')

In [6]:
file_path = files[2]
file_path

'./DORIS/795352/SDS_Other_GLO_EN_2001-01-19.pdf'

In [7]:
result = is_pdf_and_starts_with_sds(file_path)

This is an sds document.


In [8]:
result

Extraction(product_name='WACKER BS 1701', manufacturer='Wacker-Chemie GmbH', address='Hanns-Seidel-Platz 4')

In [9]:
import json
import sqlite3

In [10]:
def create_database(db_name: str):
    conn = sqlite3.connect(db_name)
    cursor = conn.cursor()
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS Extraction (
        id INTEGER PRIMARY KEY,
        product_name TEXT NOT NULL,
        manufacturer TEXT NOT NULL,
        address TEXT NOT NULL,
        UNIQUE(product_name, manufacturer)
    )
    ''')
    conn.commit()

In [11]:
def insert_extraction(cursor, extraction: Extraction):
    with sqlite3.connect(db_name) as conn:
        cursor = conn.cursor()        
        # Check if the record already exists
        cursor.execute('''
        SELECT 1 FROM Extraction
        WHERE product_name = ? AND manufacturer = ?
        ''', (extraction.product_name, extraction.manufacturer))
        
        if cursor.fetchone():
            print(f"Record with product_name='{extraction.product_name}' and manufacturer='{extraction.manufacturer}' already exists.")
            return
        
        # Insert the new record
        cursor.execute('''
        INSERT INTO Extraction (product_name, manufacturer, address)
        VALUES (:product_name, :manufacturer, :address)
        ''', extraction.dict())
        conn.commit()

db_name = 'extractions.db'
create_database(db_name)

In [19]:
# insert_extraction(db_name, result)

Record with product_name='WACKER BS 1701' and manufacturer='Wacker-Chemie GmbH' already exists.


In [32]:
len(files)

133

In [35]:
for file_path in files:
    result = is_pdf_and_starts_with_sds(file_path)  
    if result:
        insert_extraction(db_name, result)
        conn.commit()

./DORIS/795352/TDS_GLO_DE,EN_2001-03-01.pdf : File is not SDS file.
./DORIS/795352/SoC_General_GLO_DE_2004-02-01.pdf : File is not SDS file.
./DORIS/795352/SDS_Other_GLO_EN_2001-01-19.pdf : is an sds document.
Record with product_name='WACKER BS 1701' and manufacturer='Wacker-Chemie GmbH' already exists.
./DORIS/795352/SDS_Other_GLO_DE_2002-12-13.pdf : is an sds document.
Some error in parsing pdf : ./DORIS/795352/SDS_Other_GLO_DE_2002-12-13.pdf
./DORIS/795352/SDS_EU_NL_NL_2022-11-17.pdf : is an sds document.
Record with product_name='' and manufacturer='' already exists.
./DORIS/795352/SDS_Other_DE_EN_2004-02-27.pdf : is an sds document.
Some error in parsing pdf : ./DORIS/795352/SDS_Other_DE_EN_2004-02-27.pdf
./DORIS/795352/TDS_GLO_DE_2003-03-20.pdf : File is not SDS file.
./DORIS/795352/SDS_EU_DE_DE_2022-11-15.PDF : is an sds document.
Record with product_name='SILRES BS 1701' and manufacturer='Wacker Chemie AG' already exists.
./DORIS/795352/SDS_Other_GLO_FR_2000-09-12.pdf : is an 

invalid pdf header: b'\x89PNG\r'
EOF marker not found
EOF marker not found
EOF marker not found
invalid pdf header: b'PK\x03\x04\x14'
EOF marker not found
invalid pdf header: b'if (a'
EOF marker not found


Record with product_name='' and manufacturer='' already exists.
./DORIS/762154/SDS_US_JP_EN_2011-03-30.PNG : is not PDF
./DORIS/762154/SDS_US_JP_EN_2011-03-30.PNG : is an sds document.
Some error in parsing pdf : ./DORIS/762154/SDS_US_JP_EN_2011-03-30.PNG
./DORIS/762154/SDS_GHS_BB_ID_Not available.xlsx : is not PDF
./DORIS/762154/SDS_GHS_BB_ID_Not available.xlsx : is an sds document.
Some error in parsing pdf : ./DORIS/762154/SDS_GHS_BB_ID_Not available.xlsx
./DORIS/762154/GIS_GLO_EN_2018-01-14.PDF : File is not SDS file.
./DORIS/762154/SDS_Other_JP_EN_2023-09-07.txt : is not PDF
./DORIS/762154/SDS_Other_JP_EN_2023-09-07.txt : is an sds document.
Some error in parsing pdf : ./DORIS/762154/SDS_Other_JP_EN_2023-09-07.txt
./DORIS/762154/TDS_GLO_EN_2016-02-12.pdf : File is not SDS file.
./DORIS/790098/SoC_Regulatory_GLO_EN_2022-02-16.pdf : File is not SDS file.
./DORIS/790098/Certif_Halal_GLO_EN_2019-07-11.pdf : File is not SDS file.
./DORIS/790098/CoA_GLO_EN_2022-04-25.pdf : File is not S

In [38]:
with sqlite3.connect('extractions.db') as conn:
    cursor = conn.cursor()
    cursor.execute('SELECT * FROM Extraction')
    rows = cursor.fetchall()
    
rows

[(1,
  'MOUNTAIN GIV B HN2021',
  'Givaudan Fragrances Corp.',
  '300 Waterloo Valley Road, International Trade Center, MOUNT OLIVE NJ  07828, UNITED STATES OF AMERICA'),
 (2,
  'Wollastonite, Calcium silicate mineral (calcium metasilicate) surface modified',
  'NYCO Minerals Inc.',
  '803 Mountain View Drive, Willsboro NY, 12996  USA'),
 (3, 'WACKER BS 1701', 'Wacker-Chemie GmbH', 'Hanns-Seidel-Platz 4'),
 (4, '', '', ''),
 (5,
  'SILRES BS 1701',
  'Wacker Chemie AG',
  'Hanns-Seidel-Platz 4, D 81737 München'),
 (6,
  'TEGO WET 500',
  'Evonik Resource Efficiency GmbH',
  'Goldschmidtstr. 100, 45127 Essen, Deutschland'),
 (7,
  'TEGO WET 500',
  'Evonik Tego Chemie GmbH',
  'Goldschmidtstr. 100  45127 Essen'),
 (8,
  'TEGO WET 500',
  'Evonik Industries AG',
  'Goldschmidtstr. 100, D-45127 Essen'),
 (9,
  'Sodium Tolyltriazole 50% Solution',
  'NORTH Metal and Chemical Company',
  'P. O. Box 1985, 609 E. King St., York, PA USA 17405, York, PA USA 17403'),
 (10,
  'SEA BLOSSOMS',
  'F

In [45]:
def extract_data(ocr_text: str) -> Extraction:
    chat_completion = groq.chat.completions.create(
        messages=[
            {
                "role": "system",
                "content": "You are an information extractor agent. Your task is to extract specific pieces of information from the given text and outputs in JSON.. If not explicitly provided do not guess. Return empty string for respective key if data not extracted.\n"
                f" The JSON object must use the schema: {json.dumps(Extraction.model_json_schema(), indent=2)}",
            },
            {
                "role": "user",
                "content": f"Given text : \n {ocr_text}",
            },
        ],
        model="llama3-8b-8192",
        temperature=0,
        stream=False,
        response_format={"type": "json_object"},
    )
    return Extraction.model_validate_json(chat_completion.choices[0].message.content)


In [39]:
### Test doc :

In [40]:
test_pdf = PyPDFLoader('./test-doocs/SDS_US_US_EN_2021-08-23.pdf')

In [43]:
test_pages = test_pdf.load_and_split()

In [44]:
test_pages[0].page_content

'SAFETY DATA SHEET \nIn accordance with the requirements of US 29 CFR Pa rts 1910, 1915, \nand 1926.   \n \n \nMOUNTAIN GIV B HN2021 \nVersion 2.1 Revision Date 23 AUG 2021  Print Date 23 AUG 2021  \n \n \nAdministrative information:  \nReport Information: SDS_US/EN/GHS_SDS_NA_REGION/23 \nSales & Distribution Information:  MW01/FR/US13/01 \nShipping Order Information:  22,182,740/15,834,435  \n  1/13  \n SECTION 1. PRODUCT AND COMPANY IDENTIFICATION  \nProduct information \n \nSales No.  : UAK04037/00  \nMOUNTAIN GIV B HN2021  \n \nRelevant identified uses of \nthe substance or mixture \nand uses advised against  \nIntended Use  Fragrances : Perfume compound \n \nCompany  :  \nGivaudan Fragrances Corp. \n300 Waterloo Valley Road \nInternational Trade Center \nMOUNT OLIVE NJ  07828 \nUNITED STATES OF AMERICA \n \n+12152070061 \nEmergency telephone \nnumber : +18669280789  \nPlease refer to section 16 for a full list of emerg ency phone numbers. \n \n \nSECTION 2. HAZARDS IDENTIFICATION 

In [46]:
extract_data(test_pages[0].page_content)

Extraction(product_name='MOUNTAIN GIV B HN2021', manufacturer='Givaudan Fragrances Corp.', address='300 Waterloo Valley Road, International Trade Center, MOUNT OLIVE NJ  07828, UNITED STATES OF AMERICA')

In [47]:
def check_if_document_is_present_in_db(db_name, extraction):
    with sqlite3.connect(db_name) as conn:
        cursor = conn.cursor()        
        cursor.execute('''
        SELECT 1 FROM Extraction
        WHERE product_name = ? AND manufacturer = ?
        ''', (extraction.product_name, extraction.manufacturer))

        if cursor.fetchone():
            return f"Record with product_name='{extraction.product_name}' and manufacturer='{extraction.manufacturer}' already exists."


In [48]:
db_name = 'extractions.db'

In [49]:
check_if_document_is_present_in_db(db_name, extract_data(test_pages[0].page_content))

"Record with product_name='MOUNTAIN GIV B HN2021' and manufacturer='Givaudan Fragrances Corp.' already exists."

In [ ]:
metadata - version,language